In [1]:
import os

from datetime import timezone, datetime

from crypto_trades_downloader.timescaledb_util import TimeScaleDBUtil
import pandas as pd

pd.options.display.precision = 10

# PostgreSQL設定
pg_config = {
    'user': os.environ['POSTGRES_USER'],
    'password': os.environ['POSTGRES_PASSWORD'],
    'host': os.environ['POSTGRES_HOST'],
    'port': os.environ['POSTGRES_PORT'],
    'database': os.environ['POSTGRES_DATABASE']
}

dbutil = TimeScaleDBUtil(user = pg_config['user'], password = pg_config['password'], host = pg_config['host'], port = pg_config['port'], database = pg_config['database'])

In [2]:
ex_a_name = 'bitfinex2'
ex_b_name = 'bequant'
target_pair = "XRP/BTC"

table_name = dbutil.get_trade_table_name(ex_a_name, target_pair)
df_trades_a = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime < \'2021-12-31 23:59:59+00\' ORDER BY datetime')
df_trades_a = df_trades_a.rename(columns={'price': 'price_a'})
df_trades_a['exchange'] = ex_a_name

table_name = dbutil.get_trade_table_name(ex_b_name, target_pair)
df_trades_b = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime < \'2021-12-31 23:59:59+00\' ORDER BY datetime')
df_trades_b = df_trades_b.rename(columns={'price': 'price_b'})
df_trades_b['exchange'] = ex_b_name

In [3]:
df_ab_trades_nona = pd.concat([df_trades_a, df_trades_b]).sort_values('datetime', ascending=True).fillna(method='ffill').dropna().reset_index(drop=True)
df_ab_trades_nona['price_diff'] = df_ab_trades_nona['price_a'] - df_ab_trades_nona['price_b']
df_ab_trades_nona['price_diff_pct'] = df_ab_trades_nona['price_diff'] / df_ab_trades_nona['price_a'] * 100
df_ab_trades_nona['profitable'] = False
df_ab_trades_nona.loc[(df_ab_trades_nona['price_diff_pct'] > 1.0) | (df_ab_trades_nona['price_diff_pct'] < -1.0), 'profitable'] = True
df_ab_trades_nona['inverse_profitable'] = ~df_ab_trades_nona['profitable']
df_ab_trades_nona['profitable_label'] = df_ab_trades_nona['inverse_profitable'].cumsum()

In [4]:
df_ab_trades_nona_timediff = df_ab_trades_nona.sort_values('datetime', ascending=False)
df_ab_trades_nona_timediff['datetime_diff'] = -df_ab_trades_nona_timediff['datetime'].diff().dt.total_seconds()
df_ab_trades_nona_timediff.dropna(inplace=True)
df_ab_trades_nona_timediff = df_ab_trades_nona_timediff.sort_values('datetime', ascending=True).reset_index(drop=True)

In [5]:
df_ab_trades_nona_timediff_profitable = df_ab_trades_nona_timediff[df_ab_trades_nona_timediff['profitable'] == True]
df_ab_trades_nona_timediff_profitable_group = df_ab_trades_nona_timediff_profitable.groupby("profitable_label")

In [6]:
from scipy import stats
import numpy as np

print(f"{ex_a_name} vs {ex_b_name}: {target_pair}")
print(f"価格乖離が1%以上の時間は{df_ab_trades_nona_timediff_profitable['datetime_diff'].sum() / 60 / 60:.02f}時間でした\n")

print(f"\n価格乖離の分布")
for x in range(0, 101, 1):
    percentile = stats.scoreatpercentile(df_ab_trades_nona['price_diff_pct'], x)
    print(f"{x} percentile: {percentile:.3f}%")

df_time_diff = df_ab_trades_nona_timediff_profitable_group['datetime_diff'].sum()

print(f"\n価格乖離の絶対値が1%を超えた連続時間の分布")
for x in range(0, 101, 1):
    percentile = stats.scoreatpercentile(df_time_diff, x)
    print(f"{x} percentile: {percentile:.2f} sec")

bitfinex2 vs bequant: XRP/BTC
価格乖離が1%以上の時間は9.74時間でした


価格乖離の分布
0 percentile: -27.101%
1 percentile: -0.446%
2 percentile: -0.346%
3 percentile: -0.295%
4 percentile: -0.263%
5 percentile: -0.239%
6 percentile: -0.219%
7 percentile: -0.203%
8 percentile: -0.190%
9 percentile: -0.178%
10 percentile: -0.167%
11 percentile: -0.158%
12 percentile: -0.149%
13 percentile: -0.141%
14 percentile: -0.134%
15 percentile: -0.127%
16 percentile: -0.121%
17 percentile: -0.115%
18 percentile: -0.110%
19 percentile: -0.105%
20 percentile: -0.100%
21 percentile: -0.096%
22 percentile: -0.091%
23 percentile: -0.087%
24 percentile: -0.083%
25 percentile: -0.079%
26 percentile: -0.075%
27 percentile: -0.071%
28 percentile: -0.068%
29 percentile: -0.064%
30 percentile: -0.061%
31 percentile: -0.057%
32 percentile: -0.054%
33 percentile: -0.051%
34 percentile: -0.048%
35 percentile: -0.045%
36 percentile: -0.042%
37 percentile: -0.039%
38 percentile: -0.036%
39 percentile: -0.033%
40 percentile: -0.030%
41 